In [1]:
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
import configparser
from datetime import datetime
from utils import CustomDataset, SentencePairClassifier, pairing
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from bertviz import head_view
from transformers import AutoTokenizer, AutoModel, AutoConfig

from summarizer import Summarizer
import numpy as np
from torch.cuda.amp import autocast
from torch.utils.data import Dataset
import tensorflow as tf 

/userhome/30/hihung/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/userhome/30/hihung/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")
/userhome/30/hihung/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qi

# Load parameters

In [3]:
# Load parameters
print("load parameters")
config = configparser.ConfigParser()
config.read('parameter.ini')
parameters =  config['parameters']
    
summarize_question = parameters.getboolean('summarize_question')
summarize_answer = parameters.getboolean('summarize_answer')
normalize = parameters.get('normalize')
bert_model = parameters.get('bert_model')    
freeze_bert = parameters.getboolean('freeze_bert') # if True, freeze the encoder weights and only update the classification layer weights
maxlen = parameters.getint('maxlen') 
bs = parameters.getint('bs')  # batch size
iters_to_accumulate = parameters.getint('iters_to_accumulate')  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = parameters.getfloat('lr')  # learning rate
epochs = parameters.getint('epochs')  # number of training epochs
summarize = any((summarize_question, summarize_answer))


load parameters


# CounselChat Data

In [3]:
post = "I'm dealing with imposter syndrome in graduate school. I know that by all accounts\
 I am a phenomenal graduate student, and that I am well-published. I am well liked by students\
 and faculty alike. And yet I cannot shake the feeling that I'm going to be found out as a fraud.\
 How can I get over this feeling?"

In [4]:
comment1 = ['It sounds like you are on the right track. Recognizing these nagging thoughts of self- doubt\
            as "imposter syndrome" is a huge step in the right direction.  From what you have written here, it appears\
            that you are able to challenge your own thoughts and provide yourself with evidence that counteracts the\
            imposter syndrome.   Continuing to remind yourself of what you have accomplished and looking at the facts\
            at hand can help diminish doubt.  Remember, many successful people battled imposter syndrome on the way to\
            the top (and still manage it).  It might be helpful to read some of their stories so that you don\'t feel alone.\
            "The Cut" has a great article on "25 Famous Women on Imposter-Syndrome and Self-Doubt".   Business Insider has\
            a great article about men and the imposter syndrome too. Remember, if you jumped through all of the hoops to get into school and get published-- you belong. ',
           
           'Impostor syndrome is such a common experience in graduate school and other high-pressure, competitive \
           settings. I am glad to hear that you have some sense of your accomplishments, but I understand that it \
           can be quite hard to internalize and truly believe them. Here are a few suggestions for overcoming impostor syndrome:\
            Check your standards. Impostor syndrome is often connected to perfectionism, and you may be holding impossibly high\
            standards for yourself. Do you believe you need to know everything, excel at everything, or be liked by everyone in\
            order to succeed in graduate school? If so, nothing you accomplish will ever measure up to your ideals. Try shifting\
            your focus away from achievement and onto development: what are you learning about yourself and your research, and\
            how are you growing as a person and a scholar? This can help you appreciate yourself as a work in progress -- which all of us fundamentally are.\
            Own your uniqueness. It can be easy to compare yourself to peers based on number of publications, fellowships, \
            conferences, etc. But you are not a number. You are the only person in your program with your specific background,\
            interests, and point of view. The more you own what makes you special and distinct from others, the more you\'re likely\
            to feel that you have something to contribute to your program and to your field.\
            Celebrate your successes. Positive psychology has demonstrated that we can retrain our minds by paying more attention\
            to positive information. I recommend keeping a success journal and jotting down anything (large or small, concrete or\
            abstract) that you feel proud of. Aim to make a few additions a day to it. You may find that through pointing out these daily\
            victories to yourself, you start to accept more positive feedback and believe in yourself in a deeper way.\
            Talk to trusted peers and mentors. Impostor syndrome flourishes in silence. If you have an advisor or\
            colleague you can open up to, you will likely find that this person has experienced similar feelings, and that can\
            be tremendously reassuring, because it shows you that impostor syndrome is a common phenomenon, not a manifestation of your inner unworthiness.\
            A therapist can be an excellent resource on your graduate school journey to help you examine, challenge, and\
            let go of your self-doubts. I wish you the best of luck in graduate school and beyond!',
           
           'First step is to remove the label of your behavior as a syndrome and instead understand the reasons for it.\
           "Imposter syndrome\" sounds like a name someone made up to write a book and have lots of people buy it bc it\
           gives the feeling they know themselves by calling themselves this name.\
            Instead, consider your own unique qualities including your fears of being recognized as adding value to people\'s lives.\
            If you were told growing up that you\'re worthless, or if your chosen career goes against family advice and expectations,\
            or if you simply are a shy person, then these would be the starting points to understand your reluctance to believe in yourself.\
            The more you understand yourself and trust the truths you find as to who you are, the less you will feel fraudulent.\
            Good luck in your career work!',
           
           'It would be very helpful to identify with you eventual pattern where the imposter syndrome is more or less present. Are there\
           specific situations where you\'ve noticed the feelings of \"I\'m going to be found out as a fraud\" becoming more strong? It\
           seems that shaking this feeling is very important to you. In my opinion, before shaking that feeling, we need to get closer\
           to it and understand its roots. If you would like to get closer to the feeling, you might consider asking yourself questions\
           such: "What is the trigger for this feeling? How does it feel in the body? What is the thought process I engage with after\
           noticing this feeling? All the best. Rossana Mag.',
           
           'According to research, one of the ways to manage imposter feelings is to explore the cognitive distortions that contribute\
           to the negative emotions. Expert  Jessica Vanderland, Ph.D a clinical psychologist.  challenges individuals to examine the\
           bigger picture and asked the question concerning  what fact supports that you deserve to be in your role? Vanderland said,\
           \"it can help to zoom out and consider how where you are now compares to where you were last year or five years ago.\"  \
           Vanderland recommended a simple exercise of exploring how you might help a friend who undermines their achievements and\
           then apply the same supportive words to their your own narrative.',
           
           'What you described is so common for graduate students.  When I was in graduate school,\
           I felt the same way and continue to at times now even though I\'m licensed!\
           One of the things to know is that it\'s good to a certain degree to experience some self-doubt.\
           It\'s what pushes you to keep learning in your field and grow professionally.  When you are in\
           graduate school, you\'re really at the beginning of that journey so of course you aren\'t expected\
           to know everything or to feel like you have mastered what you\'ve learned.\
           Do you have a professor you are comfortable discussing this with?  Or maybe a fellow student you feel\
           a connection with?  It can be helpful to discuss these concerns with people that might understand what\
           you are going through and can offer some guidance or make you feel less alone in it.\
           Something I do to quiet that "imposter voice" is make sure I\'m not giving attention to comparisons.\
           That\'s a one way ticket to feeling incompetent.  Even comparing yourself to others in graduate school\
           is unhelpful because what we think people know and what they actually know can be distorted based on how\
           confident we feel in our own abilities.  Look at your accomplishments and the things you have truly learned.\
           You obviously put in some incredibly hard work to make it into graduate school.  How much credit are you giving\
           yourself for what you already have done or know?',
           
           '"Imposter Syndrome" is a basic effect is a dishonest cause.\
           Basically, there is a fundamental lack of honesty in some important parts of your life (my guess would be relationships)\
           that has led to this sense of dread and guilt.\
           I wonder where the dishonesty be...',
           
           'I\'m sorry you\'re feeling this way. You\'ve probably read articles already about impostor syndrome, but still didn\'t get\
           the answers you were looking for; you probably know that lots of people feel this way, that it happens to lots of successful\
           people in professional settings. Not helping, right?\
           Think about people who succeed at big tasks: an Olympic athlete has a defined goal that they need to meet: cross the finish line,\
           say, faster than anyone else. That success is remarked upon, and - until someone does the event better than they did - they are \'the best.\'\
           Grad school, however, is not so great at discrete goals and celebrations of success. You got a good grade on an assignment, but\
           what about the next one? You and 6 other people in your class probably got the same grade, so does that make you a success or average? \
           It falls to you, then, to identify ways you are sure you\'ve succeeded. If your grades and peer admiration aren\'t hallmarks of success enough,\
           perhaps pull your measuring tool inward: measure your achievements against your own progress- ie: I got a 93 on my last assignment,\
           I got a 97 this time! You might find those feelings fading soon~ '
          ]

In [5]:
comment2 = ['Watching children go through challenges in their lives is difficult. On a very basic level, There exists a primal need to \
protect them from harm. The hard part for parents is letting them feel those challenges and working through them as they \
get older. At some point, there is a moment that occurs when the role as a parent shifts. Children no longer need the \
basics (food, shelter. water, safety) as much as when they were toddlers, but rather, their needs shift to wanting more \
support, encouragement, advice, and room to make mistakes. This is where the ability to communicate with them, letting them \
direct the sails to gather the wind needed to move, is so important. Keep the lines of communication open and be available \
to give feedback when they ask for it.',
         
'Oh dear - this is becoming all too common.  I suggest you have some conversations with your girl to figure out where she \
is getting the "information" that she builds anxiety around. I see teens  who have heard teachers, parents, coaches, etc \
push the "highly selective colleges" idea on kids way too much. Teens in particular are prone to black and white thinking, \
leading them to see one poor test performance, one off day, one not-great assignment as the make-or-break task on the path \
to successful adulthood.  If that\'s the case with your daughter, she needs you to help modulate both the incoming information \
(there are nearly 5000 colleges in the US - she will get into one of them!) as well as her reaction to that information.\
As well, you and any other adults that are involved in parenting her need to discern fact from fiction.  The recent scandal \
around celebrities "buying" their kids into college is an indicator of how far off track we, as a culture, have gotten in \
terms of education.  A college education is certainly important, but we have allowed the marketing push to create so much \
stress and upset for our kids (and ourselves as parents) that they are actually less prepared to leave home and really \
engage in higher education than ever.\
You may want to include a therapist in these conversations if you get stuck, but start with a parent-child conversation. \
Help her figure out what is going on - if it is the "I have to get it right or I will have no future", calmly walk \
her through all the evidence that says that\'s just not true.  As well, let her know that being anxious now is NOT \
preparing her for a bright adulthood - it\'s preparing her for an anxious future.  She needs help figuring out how to \
balance her effort to attend to real and reasonable expectations, not over-inflated fear-based actions.',

'Your concerns about your daughter are valid and as you may think of them to be "silly" often times stress presents \
differently to those who are directly experiencing it. Stress, though a 5 letter word, is emotional and physical tension \
typically arising from the demands of life that one may believe to be complex, challenging or demanding. While you are \
afraid that she is stressing too much it is important to understanding from her perspective what it means to her to work hard in school.\
As a teenager the source of stress could function from school demands, frustrations of success vs failure, and even thoughts \
that arise about themselves, along with everyday pressures. Sometimes stress will "let up" and other times stress can become \
so overwhelming leading one to experience fight-or-flight responses as the automatic preparation that signals a sign of danger \
or threat. When this occurs natural stress can produce stress much more overwhelming and difficult to handle alone.\
Luckily, you nor your daughter have to feel alone though you are experiencing some fear and uncertainty. To assist in keeping \
stress manageable it helps to sit down with your daughter and address where she is feeling the most pressure and overwhelmingness \
and consider a schedule that provided realistic goals, rhyme and reason. Consider practicing meditation, relaxation and breathing \
exercises that serve as the natural antidote to relieve stress. If there is some hesitancy or uncomfortableness of having this \
conversation, validate your daughter\'s feeling and loop in a therapist who can further provided recommendation and stress management \
techniques along with a variety of treatment methods.\
Thank you for your courage to post this. Wishing all the best!']

# Reddit Data

In [4]:
post = "10 months ago, my husband and I eloped. We used to date in college, but we broke up and had only been seeing each other again for 5 months before we tied the knot.\
Our marriage has been really good, at least it seemed that way to me.\
Yesterday, I overheard my husband talking to his cousin. I had just come from dropping some things off for my parents, so I don’t think either of them realised I was home.\
The cousin was telling my husband how he never thought he would get married, and my husband got really quiet. He then said “he didn’t know what he was thinking” when he married me.\
His cousin asked him if he regretted it and my husband didn’t say a word. His cousins started making jokes about the situation and said something about how at least my husband gets \
to enjoy the honeymoon period sex and he said, “at least there’s that”.\
I haven’t confronted him just yet because I have the awful habit of crying whenever I’m upset. I thought I would post here to get some advice before I do. How do I proceed from here?\
TL;DR – Husband told his cousin he didn’t know what he was thinking when he married me and didn’t respond when his cousin asked him if he regretted marrying me."


In [5]:
comment1=["You need to talk to him these are some very hurtful words and if he feels this way then you need to decide what to do.",
         "Just tell him you overheard him and his cousin talking and you need to know what you meant and how he feels with your marriage and before he says anything you tell him you only want the truth. Hope all is well",
         "That absolutely sucks! It is ok for you to cry and to feel what you're feeling.\
It may be of some comfort that this comes early rather than a decade later with kids and a lot of shared assets. Or after resentment leads to bitter fights and a hateful divorce.\
Annulments were made for times like this. As much as it hurts, it may be for the best to talk to him about annulling the marriage and moving on.",
         "Husband, I heard what you said to your cousin. I need to know if you want to stay in this marriage.",
         ]

In [6]:
comment2 = ["...he threatened to shoot you and you didn’t call the cops? You’re lucky he didn’t kill you or your mother.",
           "You should have called the police. You need a restraining order.\
I’m glad you’re getting out. Very smart.",
           "When someone is holding a gun and threatens to shoot you, please leave and call the police.",
           "He sounds like a douche, but my first thought was maybe he was going to commit suicide....",
           "Wow...you were threatened by him..with a gun..and you didn't even call the police on him?\
I would just say, if this happened again, do call the police. You were betting with your own safety, I'm glad that he didn't end up shooting at you or your mom.\
I hope he doesn't contact you anymore and will leave you alone. Take care, OP. You definitely did the right choice of breaking it off with him."]

# Ranking

In [7]:
def rank_comment(post: str, raw_comments: list):
    ranking_comments = []
    for raw_comment in raw_comments:
        comment = {}
        comment['content'] = raw_comment
        comment['score'] = pairing(post, raw_comment)
        ranking_comments.append(comment)
    return ranking_comments, sorted(ranking_comments, key=lambda i: i['score'], reverse=True)
    
    

In [8]:
def summarize(text, summarizer_model, maxlen=maxlen):
    if len(text.split()) > maxlen: 
        result = summarizer_model(text, ratio=0.6, max_length=maxlen)
        #result = summarizer_model(text, max_length=maxlen)
        text = ''.join(result)
    return text

comments = comment1 + comment2
summarizer_model = Summarizer()

post = summarize(post, summarizer_model)

for c in comments:
    c = summarize(c, summarizer_model)


In [12]:

print("Post Content:")
print(post)

print("=========================================")
ranking_comments, sorted_comments = rank_comment(post, comments)
for c in sorted_comments:
    print(c['content'])
    tf.print(c['score'], output_stream=sys.stderr)
    print()
print("=========================================")

Post Content:
Yesterday, I overheard my husband talking to his cousin.


NameError: name 'np' is not defined

# Load model and run

In [ ]:
# Load model
path_to_model = "./models/roberta-large_lf_MSELoss()_lr_2e-06_val_loss_2.96311_ep_22.pt"
print("Loading the weights of the model...")
model = SentencePairClassifier(bert_model)
#tokenizer = RobertaTokenizer.from_pretrained(bert_model)
tokenizer = AutoTokenizer.from_pretrained(bert_model, use_fast=False)      

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() >= 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model.load_state_dict(torch.load(path_to_model))
model.to(device)

sentence_a = post[0]
sentence_b = comments1[0]

#model.eval()
#with torch.no_grad():
print("start visualize...")
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True, return_token_type_ids=True)
seq = inputs['input_ids']
attn_masks = inputs['attention_mask']
token_type_ids = inputs['token_type_ids']

seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
attention = model(seq,attn_masks, token_type_ids)[-1]
input_id_list = seq[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
head_view(attention, tokens)

Loading the weights of the model...
Let's use 1 GPUs!
start visualize...
